## NGT Sign Language Recognition - Model Training Pipeline

Trains a Random Forest classifier with Optuna hyperparameter tuning.

Features:

- Optuna hyperparameter optimization (100 trials)

- Cross-validation during tuning

- Saves best model with joblib

Output:

```models/random_forest.joblib``` - Trained model

In [9]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import optuna
from optuna.samplers import TPESampler

### Configuration

In [10]:
# Paths
DATA_PATH = "data/samples.csv"
MODEL_DIR = "models"
MODEL_PATH = os.path.join(MODEL_DIR, "random_forest.joblib")
REPORT_PATH = os.path.join(MODEL_DIR, "training_report.txt")

# Training settings
TEST_SIZE = 0.2
RANDOM_STATE = 42
CV_FOLDS = 5
OPTUNA_TRIALS = 100

# Optuna hyperparameter search space
PARAM_SPACE = {
    'n_estimators': (20, 300),
    'max_depth': (5, 50),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 10),
    'max_features': ['sqrt', 'log2', None]
}

print("Configuration loaded!")
print(f"  Data path: {DATA_PATH}")
print(f"  Model will be saved to: {MODEL_PATH}")

Configuration loaded!
  Data path: data/samples.csv
  Model will be saved to: models\random_forest.joblib


### Data Loading

In [11]:
# Load dataset
df = pd.read_csv(DATA_PATH)

# Separate features and labels
X = df.drop('label', axis=1)
y = df['label']

print(f"Dataset loaded!")
print(f"  Total samples: {len(df)}")
print(f"  Features: {X.shape[1]}")
print(f"  Classes: {y.nunique()} letters")

# Show samples per class
print("Samples per letter:")
sample_counts = y.value_counts().sort_index()
for label, count in sample_counts.items():
    bar = "█" * min(count // 2, 20)
    print(f"  {label}: {count:3d} {bar}")

Dataset loaded!
  Total samples: 690
  Features: 42
  Classes: 23 letters
Samples per letter:
  A:  30 ███████████████
  B:  30 ███████████████
  C:  30 ███████████████
  D:  30 ███████████████
  E:  30 ███████████████
  F:  30 ███████████████
  G:  30 ███████████████
  I:  30 ███████████████
  K:  30 ███████████████
  L:  30 ███████████████
  M:  30 ███████████████
  N:  30 ███████████████
  O:  30 ███████████████
  P:  30 ███████████████
  Q:  30 ███████████████
  R:  30 ███████████████
  S:  30 ███████████████
  T:  30 ███████████████
  U:  30 ███████████████
  V:  30 ███████████████
  W:  30 ███████████████
  X:  30 ███████████████
  Y:  30 ███████████████


In [12]:
# Split data with 80% / 20% ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    stratify=y,
    random_state=RANDOM_STATE
)

### Optuna Optimization

In [13]:
def objective(trial):
    """Optuna objective function for hyperparameter tuning."""
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', *PARAM_SPACE['n_estimators']),
        'max_depth': trial.suggest_int('max_depth', *PARAM_SPACE['max_depth']),
        'min_samples_split': trial.suggest_int('min_samples_split', *PARAM_SPACE['min_samples_split']),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', *PARAM_SPACE['min_samples_leaf']),
        'max_features': trial.suggest_categorical('max_features', PARAM_SPACE['max_features']),
        'random_state': RANDOM_STATE,
        'n_jobs': -1
    }
    
    model = RandomForestClassifier(**params)
    
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    
    return scores.mean()

In [14]:
sampler = TPESampler(seed=RANDOM_STATE)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=OPTUNA_TRIALS, show_progress_bar=True)

[I 2026-01-24 12:05:26,202] A new study created in memory with name: no-name-58dbabfb-5dd4-4878-97a8-5950bdd7bf2f
Best trial: 0. Best value: 0.978247:   1%|          | 1/100 [00:01<02:39,  1.62s/it]

[I 2026-01-24 12:05:27,816] Trial 0 finished with value: 0.9782473382473382 and parameters: {'n_estimators': 125, 'max_depth': 48, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.9782473382473382.


Best trial: 1. Best value: 0.985487:   2%|▏         | 2/100 [00:04<04:09,  2.55s/it]

[I 2026-01-24 12:05:31,021] Trial 1 finished with value: 0.9854873054873055 and parameters: {'n_estimators': 263, 'max_depth': 32, 'min_samples_split': 15, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.9854873054873055.


Best trial: 2. Best value: 0.985504:   3%|▎         | 3/100 [00:09<05:55,  3.67s/it]

...

[I 2026-01-24 12:08:23,622] Trial 97 finished with value: 0.9855036855036854 and parameters: {'n_estimators': 54, 'max_depth': 28, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': None}. Best is trial 93 with value: 0.9927764127764128.


Best trial: 93. Best value: 0.992776:  99%|█████████▉| 99/100 [02:58<00:00,  1.13it/s]

[I 2026-01-24 12:08:24,398] Trial 98 finished with value: 0.990958230958231 and parameters: {'n_estimators': 49, 'max_depth': 25, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 93 with value: 0.9927764127764128.


Best trial: 93. Best value: 0.992776: 100%|██████████| 100/100 [02:58<00:00,  1.79s/it]

[I 2026-01-24 12:08:24,990] Trial 99 finished with value: 0.990958230958231 and parameters: {'n_estimators': 31, 'max_depth': 25, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 93 with value: 0.9927764127764128.


In [15]:
best_params = study.best_params
best_cv_accuracy = study.best_value
print(f"Best CV accuracy: {best_cv_accuracy:.4f}")

Best CV accuracy: 0.9928


### Train Final Model

In [16]:
model = RandomForestClassifier(
    **best_params,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

model.fit(X_train, y_train)


,n_estimators,57
,criterion,'gini'
,max_depth,30
,min_samples_split,4
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


### Evaluate the model

In [17]:
# Predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)

# Accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {test_accuracy:.4f}")

# Average confidence for correct predictions
correct_mask = y_pred == y_test
correct_confidences = y_proba.max(axis=1)[correct_mask]
avg_confidence = correct_confidences.mean()
print(f"Average confidence (correct predictions): {avg_confidence:.4f}")


Test accuracy: 0.9928
Average confidence (correct predictions): 0.9029


### Save the model

In [18]:
os.makedirs(MODEL_DIR, exist_ok=True)
joblib.dump(model, MODEL_PATH)
print(f"Model saved to: {MODEL_PATH}")

Model saved to: models\random_forest.joblib


### Summary

In [19]:
print(f"  Best CV accuracy:   {best_cv_accuracy:.4f}")
print(f"  Test accuracy:      {test_accuracy:.4f}")
print(f"  Model saved to:     {MODEL_PATH}")

  Best CV accuracy:   0.9928
  Test accuracy:      0.9928
  Model saved to:     models\random_forest.joblib
